## 0. Libarary 불러오기 및 경로설정

In [38]:
import os
import pandas as pd
from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

import time

In [45]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = 'input/data/eval'

In [4]:
train_path = 'input/data/train'
train_image_dir_path = os.path.join(train_path, 'images')

Dataset 생성

모든 train data의 path를 가져와 라벨링 진행

In [5]:
def search(dirname, result): # 하위 목록의 모든 파일을 찾는 함수
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            if filename[0] == '.': # .으로 시작하는 애들 거름
                continue
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename, result)
            else:
                ext = os.path.splitext(full_filename)[-1] # 확장자 체크
                if ext:
                    result.append(full_filename)
        
    except PermissionError:
        print('Permission Error')
        pass

In [6]:
all_path = list()
search(train_image_dir_path, all_path)

In [7]:
all_path = sorted(all_path)

라벨링을 하는 함수입니다. 조건에 따라 label에 숫자를 더해주는 식으로 만들었습니다.

In [8]:
def labeling(name):
    label = 0
    info, mask_type = name.split('/')[-2:]
    info = info.split('_')
    gender, age = info[1], int(info[3])
    
    # 마스크 구별
    if 'incorrect' in mask_type:
        label += 6
    elif 'normal' in mask_type:
        label += 12
    
    # gender 구별
    if gender == 'female':
        label += 3
    
    # 나이 구별
    if 30 <= age and age < 60:
        label += 1
    elif age >= 60:
        label += 1
    
    return label

In [9]:
train_path_label = pd.DataFrame(all_path, columns = ['path'])

train_path_label['label'] = train_path_label['path'].map(lambda x: labeling(x))
train_path_label

,path,label
0,input/data/train/images/000001_female_Asian_45...,10
1,input/data/train/images/000001_female_Asian_45...,4
2,input/data/train/images/000001_female_Asian_45...,4
3,input/data/train/images/000001_female_Asian_45...,4
4,input/data/train/images/000001_female_Asian_45...,4
...,...,...
18895,input/data/train/images/006959_male_Asian_19/m...,0
18896,input/data/train/images/006959_male_Asian_19/m...,0
18897,input/data/train/images/006959_male_Asian_19/m...,0
18898,input/data/train/images/006959_male_Asian_19/m...,0


## 1. Test Dataset 정의

In [52]:
class CustomDataset(Dataset):
    def __init__(self, img_paths_label, transform):
        print(img_paths_label)
        self.X = img_paths_label['path']
        self.y = img_paths_label['label']
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.X.iloc[index])
        label = self.y.iloc[index]
        
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)

    def __len__(self):
        return len(self.X)

In [23]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

train, valid를 나누는 부분입니다.

label의 비율을 유지하면서 나눴습니다.

In [24]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train_path_label, test_size=0.2,
                               shuffle=True, stratify=train_path_label['label'],
                               random_state=34)

dataloader를 정의했습니다. batchsize는 64로 했고 shuffle을 했습니다.

In [25]:
BATCH_SIZE = 64

In [53]:
train_dataset = CustomDataset(train, transform)

train_dataloader = DataLoader(train_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True
                             )

                                                    path  label
3069   input/data/train/images/001058_female_Asian_28...      3
18769  input/data/train/images/006927_male_Asian_19/m...      0
12116  input/data/train/images/004070_female_Asian_55...     16
11234  input/data/train/images/003766_male_Asian_38/n...     13
12897  input/data/train/images/004320_female_Asian_58...      4
...                                                  ...    ...
16674  input/data/train/images/006339_female_Asian_18...      9
12964  input/data/train/images/004333_male_Asian_57/i...      7
4120   input/data/train/images/001239_male_Asian_25/m...      0
351    input/data/train/images/000078_female_Asian_55...      4
4749   input/data/train/images/001378_female_Asian_55...      4

[15120 rows x 2 columns]


In [27]:
valid_dataset = CustomDataset(valid, transform)

valid_dataloader = DataLoader(valid_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

dataloader는 [batchsize, channel, height, wide]를 출력해줍니다.

In [28]:
next(iter(train_dataloader))[0].shape

torch.Size([64, 3, 512, 384])

In [29]:
next(iter(valid_dataloader))[0].shape

torch.Size([64, 3, 512, 384])

## 2. Model 정의

In [ ]:
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(MyModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## 1. Model 정의

모델은 pretrain된 resnet18을 가져왔습니다. 이 모델의 마지막 fc층만 저희의 과제인 18개의 class로 변경해줍니다.

In [33]:
resnet18 = torchvision.models.resnet18(pretrained=True)

In [34]:
import math

OUTPUT_CLASS_NUM = 18
resnet18.fc = torch.nn.Linear(in_features=512, out_features=OUTPUT_CLASS_NUM, bias=True) # output 18개로

# xavier uniform
torch.nn.init.xavier_uniform_(resnet18.fc.weight)
stdv = 1. / math.sqrt(resnet18.fc.weight.size(1))
resnet18.fc.bias.data.uniform_(-stdv, stdv)

resnet18.fc.weight.shape[0]

18

In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

아래 대부분의 코드가 부스트캠프에서 학습 자료나 과제로 제공받았던 코드를 거의 그대로 사용했습니다.

설명도 주석도 잘 달려 있어서 그대로 가져왔습니다.

epoch는 5, lr은 0.0001로 주었습니다.

추후에 lr scheduler로 lr을 변경해보는 방법도 좋을 것 같습니다.

In [36]:
resnet18.to(device)

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 5 # 학습 때 mnist train data set을 얼마나 많이 학습할 지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross Entropy Loss를 objective function으로 사용
optimizer = torch.optim.Adam(resnet18.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train": train_dataloader,
    "test": valid_dataloader,
}

In [39]:
best_test_accuracy = 0.
best_test_loss = 9999.
start = time.time()  # 시작 시간 저장

for epoch in range(NUM_EPOCH):
    for phase in ["train", "test"]:
        running_loss = 0.
        running_acc = 0.
        # 네트워크 모델을 train 모드로 두어 gradient를 계산하고, 
        # 여러 sub module (배치 정규화, 드롭아웃 등)이 train_mode로 작동할 수 있게 함.
        if phase == "train":
            resnet18.train()
        # 네트워크 모델을 eval 모드로 두어 여러 sub module들이 eval mode로 작동할 수 있게 함.
        elif phase == "test":
            resnet18.eval()
            
        for ind, (images, labels) in enumerate(dataloaders[phase]):
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함.
            
            # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
            with torch.set_grad_enabled(phase == "train"):
                logits = resnet18(images)
                # 모델에서 linear 값으로 나오는 예측 값([0.9, 1.2, 3.2, 0.1, -0.1, ...])에서 최대 output index를 찾아 예측 레이블([2])로 변경함
                _, preds = torch.max(logits, 1)
                loss = loss_fn(logits, labels)
                
                if phase == "train":
                    loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient를 계산
                    optimizer.step() # 계산된 gradient를 가지고 모델 업데이트
                    
            running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
            running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
            
        # 한 epoch이 모두 종료되었을 때,
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)
        
        seconds = int(time.time() - start)
        print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss: {epoch_loss:.3f}, 평균 Accuracy: {epoch_acc:.3f}")
        print(f"소요 시간: {seconds // 60}분 {seconds % 60}초")  # 현재시각 - 시작시간 = 실행 시간
        
        # phase가 test일 때
        if phase == "test":
            # best accuracy 계산
            if best_test_accuracy < epoch_acc:
                best_test_accuracy = epoch_acc
            # best loss 계산
            if best_test_loss > epoch_loss:
                best_test_loss = epoch_loss
                
seconds = int(time.time() - start)
print("학습 종료!")
print(f"최고 accuracy: {best_test_accuracy}, 최고 낮은 loss: {best_test_loss}")
print(f"소요 시간: {seconds // 60}분 {seconds % 60}초")  # 현재시각 - 시작시간 = 실행 시간

현재 epoch-0의 train-데이터 셋에서 평균 Loss: 0.296, 평균 Accuracy: 0.917
소요 시간: 2분 28초
현재 epoch-0의 test-데이터 셋에서 평균 Loss: 0.068, 평균 Accuracy: 0.978
소요 시간: 2분 55초
현재 epoch-1의 train-데이터 셋에서 평균 Loss: 0.032, 평균 Accuracy: 0.993
소요 시간: 5분 23초
현재 epoch-1의 test-데이터 셋에서 평균 Loss: 0.049, 평균 Accuracy: 0.982
소요 시간: 5분 49초
현재 epoch-2의 train-데이터 셋에서 평균 Loss: 0.009, 평균 Accuracy: 0.999
소요 시간: 8분 16초
현재 epoch-2의 test-데이터 셋에서 평균 Loss: 0.022, 평균 Accuracy: 0.994
소요 시간: 8분 42초
현재 epoch-3의 train-데이터 셋에서 평균 Loss: 0.005, 평균 Accuracy: 0.999
소요 시간: 11분 9초
현재 epoch-3의 test-데이터 셋에서 평균 Loss: 0.019, 평균 Accuracy: 0.994
소요 시간: 11분 35초
현재 epoch-4의 train-데이터 셋에서 평균 Loss: 0.002, 평균 Accuracy: 1.000
소요 시간: 14분 2초
현재 epoch-4의 test-데이터 셋에서 평균 Loss: 0.015, 평균 Accuracy: 0.996
소요 시간: 14분 28초
학습 종료!
최고 accuracy: 0.9955026507377625, 최고 낮은 loss: 0.015069482430369745
소요 시간: 14분 28초


## 3. Inference

In [58]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

ans_path_label = pd.DataFrame(image_paths, columns = ['pred'])

dataset = CustomDataset(ans_path_label, transform)

valid_testing_dataloader = DataLoader(dataset, shuffle=False)

ans_path_label['pred'] = dataset['pred'].map(lambda x: labeling(x))
# ans_path_label



check_eval_df = check_eval(valid, valid_testing_dataloader, resnet18, device)
# check_eval_df

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
# model = MyModel(num_classes=18).to(device)
model = torchvision.models.resnet18(pretrained=True)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

                                                    pred
0      input/data/eval/images/cbc5c6e168e63498590db46...
1      input/data/eval/images/0e72482bf56b3581c081f7d...
2      input/data/eval/images/b549040c49190cedc413277...
3      input/data/eval/images/4f9cb2a045c6d5b9e50ad34...
4      input/data/eval/images/248428d9a4a5b6229a7081c...
...                                                  ...
12595  input/data/eval/images/d71d4570505d6af8f777690...
12596  input/data/eval/images/6cf1300e8e218716728d582...
12597  input/data/eval/images/8140edbba31c3a824e817e6...
12598  input/data/eval/images/030d439efe6fb5a7bafda45...
12599  input/data/eval/images/f1e0b9594ae9f72571f0a9d...

[12600 rows x 1 columns]


KeyError: 'path'